<a href="https://colab.research.google.com/github/mucahitrtn/Convert-PascalVoc-to-Yolo-format/blob/main/PascalVocToYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  4 22:01:24 2022

@author: mucahitrtn
"""

from bs4 import BeautifulSoup
import glob
import pandas as pd
import numpy as np


class Pascal_to_yolo_format:
  
  def __init__(self, pathSrc, pathDest):
    self.pathSrc  = pathSrc
    self.pathDest = pathDest
    
  def listToPd(self, xmlObj):
    elements = ["class", "xmin", "ymin", "xmax", "ymax", "name"]
    elements_pd = ["class", "xmin", "ymin", "xmax", "ymax", "ClassName"]
    df = pd.DataFrame(columns=elements_pd)

    for i in xmlObj:
      temp = []
      for element in elements:
        temp_idx = i.find(element)
        temp_idx= temp_idx.prettify()
        temp_idx_arr= temp_idx.split()
        temp.append(temp_idx_arr[1])
      df_length = len(df)
      df.loc[df_length] = temp
    return df

  def parsePath(self, pathXml):
      arr_splitted = pathXml.split('/')
      last = arr_splitted[-1]
      last_splitted = last.split('.')
      imgName = last_splitted[0]
      return imgName
      
  def createYoloTxt(self, yoloFormat_df, imgName):
    np.savetxt(r''+self.pathDest+'/'+imgName+'.txt', yoloFormat_df.values, fmt='%d %f %f %f %f')  

  def pascalVocToYolo(self, df, imgWidth, imgHeight):
    yolo_idx = ["class", "x_center","y_center","width", "height"]
    df_yolo = pd.DataFrame(columns= yolo_idx)

    for idx in range(len(df)):
      x_center = float(((float(df.iloc[idx]["xmin"]) + float(df.iloc[idx]["xmax"]))/2)/imgWidth)
      y_center = float(((float(df.iloc[idx]["ymin"]) + float(df.iloc[idx]["ymax"]))/2)/imgHeight)
      width  = float((float(df.iloc[idx]["xmax"]) - float(df.iloc[idx]["xmin"]))/imgWidth) 
      height = float((float(df.iloc[idx]["ymax"]) - float(df.iloc[idx]["ymin"]))/imgHeight)

      temp_arr = [int(df.iloc[idx][0]),x_center, y_center, width, height]
      temp = (temp_arr)
      df_length = len(df_yolo)
      df_yolo.loc[df_length] = temp
 
    return df_yolo  

  def convert(self):
    xmlFiles = glob.glob(self.pathSrc)

    for i in xmlFiles:
      with open(i, 'r') as f:
        data = f.read()
      imgName = self.parsePath(i)   
      sign = BeautifulSoup(data, "xml")
      imgWidth  = sign.find('width').prettify().split()
      imgHeight = sign.find('height').prettify().split() 
      objects = sign.find_all('object')
      vocFormat_df  = self.listToPd(objects)
      yoloFormat_df = self.pascalVocToYolo(vocFormat_df, float(imgWidth[1]), float(imgHeight[1]))
      self.createYoloTxt(yoloFormat_df, imgName)  


pathSrc = "/content/op/traffic_signs_dataset_v1/train/*.xml"
pathDest = "/content/destFold"

cvt = Pascal_to_yolo_format(pathSrc, pathDest)
cvt.convert()

